<h1> 2a. Getting started with TensorFlow </h1>

This notebook is Lab2a of CPB 102, Google's course on Machine Learning using Cloud ML.

In this notebook, we will create a machine learning model using tf.learn and evaluate its performance.  The dataset is rather small (7700 samples), so we can do it all in-memory.  We will also simply pass the raw data in as-is. 

In [11]:
import datalab.bigquery as bq
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

Read data created in Lab1a.

In [12]:
def read_dataset(filename):
  return pd.read_csv(filename, header=None, names=['pickuplon','pickuplat','dropofflon','dropofflat','passengers','fare_amount'])

df_train = read_dataset('../lab1a/taxi-train.csv')
df_valid = read_dataset('../lab1a/taxi-valid.csv')
df_test = read_dataset('../lab1a/taxi-test.csv')
df_train[:5]

,pickuplon,pickuplat,dropofflon,dropofflat,passengers,fare_amount
0,-74.003375,40.743642,-73.993685,40.728487,1,6.9
1,-73.990978,40.745340,-73.979782,40.752980,2,5.7
2,-74.011400,40.701761,-73.978215,40.764434,1,20.0
3,-73.970957,40.758473,-73.979440,40.759398,1,4.9
4,-73.990905,40.749596,-73.974205,40.756526,1,9.5


Setup a couple of variables based on the above dataset

In [13]:
FEATURE_COLS = np.arange(0,5)
TARGET_COL   = 'fare_amount'

<h3> Linear Regression with tf.learn Estimators framework </h3>

In [14]:
tf.logging.set_verbosity(tf.logging.INFO)
predictors = df_train.iloc[:,FEATURE_COLS].values # np.ndarray
targets = df_train[TARGET_COL].values
features = tf.contrib.learn.infer_real_valued_columns_from_input(predictors)
shutil.rmtree('taxi_model', ignore_errors=True) # start fresh each time
model = tf.contrib.learn.LinearRegressor(feature_columns=features, model_dir='taxi_model')
model = model.fit(predictors, targets, steps=1000)

Explicitly set `enable_centered_bias` to 'True' if you want to keep existing behaviour.
INFO:tensorflow:Using config: {'task': 0, 'save_summary_steps': 100, 'keep_checkpoint_max': 5, '_is_chief': True, 'save_checkpoints_secs': 600, 'evaluation_master': '', 'tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, 'master': '', 'keep_checkpoint_every_n_hours': 10000, '_job_name': None, 'cluster_spec': None, 'tf_random_seed': None, 'num_ps_replicas': 0}
INFO:tensorflow:Setting feature info to TensorSignature(dtype=tf.float64, shape=TensorShape([Dimension(None), Dimension(5)]), is_sparse=False)
INFO:tensorflow:Setting targets info to TensorSignature(dtype=tf.float64, shape=TensorShape([Dimension(None)]), is_sparse=False)
INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=5, default_value=None, dtype=tf.float64, normalizer=None)
INFO:tensorflow:Create CheckpointSaverHook
INFO:tensorflow:loss = 214.195, step = 1
INFO:tensorflow:Saving checkpoin

Evaluate on the validation data (we should defer using the test data to after we have selected a final model).

In [15]:
def compute_rmse(actual, predicted):
  return np.sqrt(np.mean((actual-predicted)**2))

def print_rmse(model):
  print "Train RMSE = {0}".format(compute_rmse(df_train[TARGET_COL], model.predict(df_train.iloc[:,FEATURE_COLS].values)))
  print "Valid RMSE = {0}".format(compute_rmse(df_valid[TARGET_COL], model.predict(df_valid.iloc[:,FEATURE_COLS].values)))

print_rmse(model)

Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.
INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=5, default_value=None, dtype=tf.float64, normalizer=None)
INFO:tensorflow:Loading model from checkpoint: taxi_model/model.ckpt-1000-?????-of-00001.
Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.
INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=5, default_value=None, dtype=tf.float64, normalizer=None)
INFO:tensorflow:Loading model from checkpoint: taxi_model/model.ckpt-1000-?????-of-00001.


Train RMSE = 9.43413080275
Valid RMSE = 9.67350231216


This is nowhere near our benchmark (RMSE of $6 or so on this data), but it serves to demonstrate what TensorFlow code looks like.  Let's use this model for prediction.

In [16]:
ROWS = np.arange(10,15)
inputs = df_test.iloc[ROWS,FEATURE_COLS]
trainedmodel = tf.contrib.learn.LinearRegressor(
  feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(inputs.values),
  model_dir='taxi_model')
print trainedmodel.predict(inputs.values)
print df_test.iloc[ROWS,:]

Explicitly set `enable_centered_bias` to 'True' if you want to keep existing behaviour.
INFO:tensorflow:Using config: {'task': 0, 'save_summary_steps': 100, 'keep_checkpoint_max': 5, '_is_chief': True, 'save_checkpoints_secs': 600, 'evaluation_master': '', 'tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, 'master': '', 'keep_checkpoint_every_n_hours': 10000, '_job_name': None, 'cluster_spec': None, 'tf_random_seed': None, 'num_ps_replicas': 0}
Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.
INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=5, default_value=None, dtype=tf.float64, normalizer=None)
INFO:tensorflow:Loading model from checkpoint: taxi_model/model.ckpt-1000-?????-of-00001.


[ 11.19960403  11.23214722  11.29799271  11.23215008  11.16654587]
    pickuplon  pickuplat  dropofflon  dropofflat  passengers  fare_amount
10 -73.977244  40.760012  -73.971044   40.764440           2          5.3
11 -73.964390  40.765414  -73.949860   40.772305           3          7.0
12 -74.011712  40.716677  -73.982800   40.739303           5         15.3
13 -73.982994  40.739079  -73.985207   40.747837           3          6.5
14 -73.992609  40.724428  -73.991879   40.764680           1         11.3


This explains why the RMSE was so high -- the model essentially predicts $11 for every trip.  Would a more complex model help? Let's try using a deep neural network.  The code to do this is quite straightforward as well.

<h3> Deep Neural Network regression </h3>

In [17]:
shutil.rmtree('taxi_model', ignore_errors=True) # start fresh each time
model = tf.contrib.learn.DNNRegressor(feature_columns=features, hidden_units=[128, 100, 8], model_dir='taxi_model')
model = model.fit(predictors, targets, steps=1000)
print_rmse(model)

Explicitly set `enable_centered_bias` to 'True' if you want to keep existing behaviour.
INFO:tensorflow:Using config: {'task': 0, 'save_summary_steps': 100, 'keep_checkpoint_max': 5, '_is_chief': True, 'save_checkpoints_secs': 600, 'evaluation_master': '', 'tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, 'master': '', 'keep_checkpoint_every_n_hours': 10000, '_job_name': None, 'cluster_spec': None, 'tf_random_seed': None, 'num_ps_replicas': 0}
INFO:tensorflow:Setting feature info to TensorSignature(dtype=tf.float64, shape=TensorShape([Dimension(None), Dimension(5)]), is_sparse=False)
INFO:tensorflow:Setting targets info to TensorSignature(dtype=tf.float64, shape=TensorShape([Dimension(None)]), is_sparse=False)
INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=5, default_value=None, dtype=tf.float64, normalizer=None)
INFO:tensorflow:Create CheckpointSaverHook
INFO:tensorflow:loss = 319.3, step = 1
INFO:tensorflow:Saving checkpoints

Train RMSE = 9.43409402844
Valid RMSE = 9.67363336918


We are not beating our benchmark with either model ... what's up?  Well, we may be using TensorFlow for Machine Learning, but we are not yet using it well.  That's what the rest of this course is about!

But, for the record, let's say we had to choose between the two models. We'd choose the one with the lower validation error. Finally, we'd measure the RMSE on the test data with this chosen model.

In [18]:
print "Test RMSE = {0}".format(compute_rmse(df_test[TARGET_COL], model.predict(df_test.iloc[:,FEATURE_COLS].values)))

Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.
INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=5, default_value=None, dtype=tf.float64, normalizer=None)
INFO:tensorflow:Loading model from checkpoint: taxi_model/model.ckpt-1000-?????-of-00001.


Test RMSE = 8.68156893269


<h2> Benchmark dataset </h2>

Let's do this on the benchmark dataset.

In [19]:
import datalab.bigquery as bq
import numpy as np
import pandas as pd


def create_query(phase, EVERY_N):
  """
  phase: 1=train 2=valid
  """
  base_query = """
SELECT
  DAYOFWEEK(pickup_datetime)*1.0 AS dayofweek,
  HOUR(pickup_datetime)*1.0 AS hourofday,
  pickup_longitude, pickup_latitude, 
  dropoff_longitude, dropoff_latitude,
  passenger_count*1.0 AS passenger_count,
  (tolls_amount + fare_amount) as fare_amount
FROM
  [nyc-tlc:yellow.trips]
WHERE
    trip_distance > 0
    AND fare_amount >= 2.5
    AND pickup_longitude > -78
    AND pickup_longitude < -70
    AND dropoff_longitude > -78
    AND dropoff_longitude < -70
    AND pickup_latitude > 37
    AND pickup_latitude < 45
    AND dropoff_latitude > 37
    AND dropoff_latitude < 45
    AND passenger_count > 0 
  """

  if EVERY_N == None:
    if phase < 2:
      # training
      query = "{0} AND ABS(HASH(pickup_datetime)) % 4 < 2".format(base_query)
    else:
      query = "{0} AND ABS(HASH(pickup_datetime)) % 4 == {1}".format(base_query, phase)
  else:
      query = "{0} AND ABS(HASH(pickup_datetime)) % {1} == {2}".format(base_query, EVERY_N, phase)
    
  return query


query = create_query(2, 100000)
df_test = bq.Query(query).to_dataframe()
print "Test RMSE = {0}".format(compute_rmse(df_test[TARGET_COL], model.predict(df_test.iloc[:,FEATURE_COLS].values)))

Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.
INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=5, default_value=None, dtype=tf.float64, normalizer=None)
INFO:tensorflow:Loading model from checkpoint: taxi_model/model.ckpt-1000-?????-of-00001.


Test RMSE = 10.344307593


Copyright 2016 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License